In [ ]:
# 라이브러리
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import time

In [ ]:
# 함수 정의 및 카테고리 코드 입력

def prep(str):
    return re.sub("[^0-9a-zA-Z가-힣 ]","",str).strip()

category_code = "100000100010008"

headers = {
    "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36"
}



In [ ]:
# 상품 리스트 수집


home_url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo={category_code}&fltDispCatNo=&prdSort=01&pageIdx=1&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=&bShowCnt=&cShowCnt=&trackingCd=Cat{category_code}_Small"
home_response = requests.get(home_url,headers = headers)
home_soup = BeautifulSoup(home_response.text, "html.parser")
url = f"https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo={category_code}&isLoginCnt=4&aShowCnt=0&bShowCnt=0&cShowCnt=0&gateCd=Drawer&trackingCd=Cat{category_code}_MID"
total_count = int(home_soup.select_one("p.cate_info_tx > span").get_text().replace(",","").strip())
prd_url_list = []


for page_num in range(total_count//100 +1):
    data ={
        "pageIdx" : str(page_num+1),
        "rowsPerPage" : 70
    }


    response = requests.get(url, headers = headers, params = data)
    soup = BeautifulSoup(response.text, "html.parser")
    prd_url = [x["href"] for x in soup.select("div.prd_info > a")]
    prd_url_list.extend(prd_url)
    


In [ ]:
home_url

In [ ]:
# 상품 정보 수집

start_time = datetime.now()
result = []
count = 0 
input_url_list = prd_url_list[len(result):]
# while len(result) < len(prd_url_list):
#     input_url_list = prd_url_list[len(result):]
for each_url in input_url_list:
#     try:
    #기본 정보
    time.sleep(0.3)
    basic_response = requests.get(each_url,headers = headers)
    basic_soup = BeautifulSoup(basic_response.text, "html.parser")
    prd_no = re.search("goodsNo=(.*)&dispCatNo", each_url).group(1)

    brand = basic_soup.select_one("a.pd_arrow_link").get_text().strip()
    prd_name = basic_soup.select_one("#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_name").get_text().strip()

    # 기획이라는 단어 들어가면 스킵
    if "기획" in prd_name:
        continue

    #가격 정보 수집
    try:
        price = int(basic_soup.select_one("#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span.price-1 > strike").get_text().replace(",","").strip())
    except:
        price = int(basic_soup.select_one("#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span > strong").get_text().replace(",","").strip())

    score = basic_soup.select_one("#repReview > b").get_text().strip()
    
    img_url = basic_soup.select_one("#mainImg")["src"]

    #상세정보
    time.sleep(0.3)
    detail_response = requests.post("https://www.oliveyoung.co.kr/store/goods/getGoodsArtcAjax.do",headers = headers, data = {"goodsNo" : prd_no})
    detail_soup = BeautifulSoup(detail_response.text, "html.parser")
    
    try:
        spec = prep(detail_soup.select_one("dl.detail_info_list:nth-child(2) > dd").get_text())
        ingredient = prep(detail_soup.select_one("dl.detail_info_list:nth-child(7) > dd").get_text())
    except:
        spec = ""
        ingredient = ""
    # 요약리뷰
    summary_review_url = "https://www.oliveyoung.co.kr/store/goods/getGdasSummaryAjax.do"
    summary_review_data = {
        "goodsNo": prd_no,
        "gdasSeq": "",
        "fileSeq": "",
        "gdasSort": "05",
        "itemNo": "all_search",
        "optionValue": ""
    }
    time.sleep(0.3)
    summary_review_response = requests.post(summary_review_url, headers = headers, data=summary_review_data)
    summary_review_soup = BeautifulSoup(summary_review_response.text, "html.parser")

    #총 리뷰 개수 수집
    try:
        total_review_cnt = int(summary_review_soup.select_one("p.total > em").get_text().replace(",","").strip())
    except:
        total_review_cnt = 0

    try:
        score_5 = int(prep(summary_review_soup.select("ul.graph_list > li > span.per")[0].get_text()))/100
        score_4 = int(prep(summary_review_soup.select("ul.graph_list > li > span.per")[1].get_text()))/100
        score_3 = int(prep(summary_review_soup.select("ul.graph_list > li > span.per")[2].get_text()))/100
        score_2 = int(prep(summary_review_soup.select("ul.graph_list > li > span.per")[3].get_text()))/100
        score_1 = int(prep(summary_review_soup.select("ul.graph_list > li > span.per")[4].get_text()))/100

    except:
        score_5 = 0
        score_4 = 0
        score_3 = 0
        score_2 = 0
        score_1 = 0

    try:
        skin_dry = int(prep(summary_review_soup.select("ul.list > li > em.per")[0].get_text()))/100
        skin_complex = int(prep(summary_review_soup.select("ul.list > li > em.per")[1].get_text()))/100
        skin_oil = int(prep(summary_review_soup.select("ul.list > li > em.per")[2].get_text()))/100
        for_moisture = int(prep(summary_review_soup.select("ul.list > li > em.per")[3].get_text()))/100
        for_relax = int(prep(summary_review_soup.select("ul.list > li > em.per")[4].get_text()))/100
        for_whitening = int(prep(summary_review_soup.select("ul.list > li > em.per")[5].get_text()))/100
        irritations_row = int(prep(summary_review_soup.select("ul.list > li > em.per")[6].get_text()))/100
        irritations_norm = int(prep(summary_review_soup.select("ul.list > li > em.per")[7].get_text()))/100
        irritations_high = int(prep(summary_review_soup.select("ul.list > li > em.per")[8].get_text()))/100

    except:
        skin_dry = 0
        skin_complex = 0
        skin_oil = 0
        for_moisture = 0
        for_relax = 0
        for_whitening = 0
        irritations_row = 0
        irritations_norm = 0
        irritations_high = 0




    data_prd = {
        "00.상품코드" : prd_no,
        "00.상품_URL" : each_url,
        "00.이미지_URL" : img_url,
        "01.브랜드" : brand,
        "02.상품명" : prd_name,
        "03.가격" : price,
        "04.제품 주요 사양" : spec,
        "05.모든 성분" : ingredient,
        "06.총 평점" : score,
        "07.리뷰 개수" : total_review_cnt,
        "08_1.별점 1점" : score_1,
        "08_2.별점 2점" : score_2,
        "08_3.별점 3점" : score_3,
        "08_4.별점 4점" : score_4,
        "08_5.별점 5점" : score_5,
        "09_1.피부타입_건성" : skin_dry,
        "09_2.피부타입_복합성" : skin_complex,
        "09_3.피부타입_지성" : skin_oil,
        "10_1.피부고민_보습" : for_moisture,
        "10_2.피부고민_진정" : for_relax,
        "10_3.피부고민_주름/미백" : for_whitening,
        "11_1.피부자극_없음" : irritations_row,
        "11_2.피부자극_보통" : irritations_norm,
        "11_3.피부자극_있음" : irritations_high

    }

    result.append(data_prd)
    count+=1
#         except Exception as e:
#             print(e)
#             break
            
result_df = pd.DataFrame(result)
result_df.to_csv("basic_data_img.csv")

end_time = datetime.now()
print(end_time-start_time)

In [ ]:
len(result)

In [ ]:
"done"

In [ ]:
# 리뷰 수집

start_time = datetime.now()
cont = []
exep = []
review_input = result_df[["00.상품코드", "02.상품명", "07.리뷰 개수"]]
review_result = []

for idx in range(len(review_input)):
    prd_name = review_input["02.상품명"].iloc[idx]
    if "기획" in prd_name:
        continue
        
    else:
        cont.append(prd_name)

    prd_no = review_input["00.상품코드"].iloc[idx]
    review_total_count = review_input["07.리뷰 개수"].iloc[idx]

    if review_total_count != 0:

        if review_total_count> 500:
            review_total_count = 500

        for count in range(review_total_count//10):
            review_url = "https://www.oliveyoung.co.kr/store/goods/getGdasNewListJson.do"
            params = {
                "goodsNo" : prd_no,
                "itemNo": "all_search",
                "pageIdx": count+1,
                "cTypeLength": "0", 
            }

            headers = {
                "Connection": "keep-alive",
                "Cookie": 'JSESSIONID=oEQGrmL51sra7OlKRXJaob7VSXv69oiJwRX2zkp5Z556EDCD7aamDWxZn5fM77ch.cE95bWFsbF9kb21haW4vb3ltcHByZDQx; RB_PCID=1651071871849360946; sch_check=yes; _gcl_au=1.1.1889217926.1651071873; PCID=16510718734998617535790; _gid=GA1.3.587031691.1651071874; _trs_id=eY54%3E61542551%3F46; oliveyoung_CID=ee4589fd664344ed8561293ccb16afec; EG_GUID=4d9a4f6f-8823-4625-a82e-79292069bbd7; _fbp=fb.2.1651071874267.525729973; WMONID=goRlNLAdaSV; _trs_sid=G%5B64664145531%3D%5Bg%5B76%3C43760773%3D64; _trs_flow=; _gat_UA-92021806-9=1; _gat_UA-181867310-1=1; _gat=1; productHistory=[{"goodsNo":"A000000162254","viewCount":15},{"goodsNo":"A000000164741","viewCount":1},{"goodsNo":"A000000163494","viewCount":1},{"goodsNo":"A000000016152","viewCount":1},{"goodsNo":"A000000155253","viewCount":1}]; _ga_GMKKBJ29S2=GS1.1.1651395570.10.1.1651395715.48; wcs_bt=s_3ee47970f314:1651395715; cto_bundle=rjnsSF82dTg2M3lGQSUyQlhxNDZiR3l3eEE0MHB4TllNNUJPa09qY1c5TkQ1NzNBeGYzUjJ0dGpFN2FOSzJpMmpqa3FncE11JTJGOEVFcXBBJTJCNjZ2SFNLMVR1V2o0V3NqYXpoMzJyUENwdjA2d1RXJTJGdkRINzBpM0dzVE0wSGFuSkY3RWJCTnhBQSUyRlRmJTJGJTJGZlQ3OEpPSTFDM0M5ZlR4ZyUzRCUzRA; RB_SSID=lmKduZkgpl; _dd_s=rum=0&expire=1651396623362; _ga_TTX3Z62VLN=GS1.1.1651395570.11.1.1651395723.0; _ga_P9V7281JHW=GS1.1.1651395714.10.1.1651395723.51; _ga=GA1.1.1645121991.1651071874',
                "DNT": "1",
                "Host": "www.oliveyoung.co.kr",
                "Referer": f"https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={prd_no}&dispCatNo={category_code}&trackingCd=Cat{category_code}_Small",
                "sec-ch-ua": '" Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"',
                "sec-ch-ua-mobile": "?0",
                "sec-ch-ua-platform": "Windows",
                "Sec-Fetch-Dest": "empty",
                "Sec-Fetch-Mode": "cors",
                "Sec-Fetch-Site": "same-origin",
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36",
                "x-datadog-origin": "rum",
                "x-datadog-parent-id": "2684032138873698185",
                "x-datadog-sampled": "1",
                "x-datadog-sampling-priority": "1",
                "x-datadog-trace-id": "664689280025308362",
                "X-Requested-With": "XMLHttpRequest",

            }
            review_response = requests.get(review_url, headers = headers, params = params)
            for each_review in range(len(review_response.json()["gdasList"])):
                try:
                    reviewr_name = review_response.json()["gdasList"][each_review]["mbrNickNm"]
                    if reviewr_name == None:
                        reviewr_name == review_response.json()["gdasList"][each_review]["mbrId"]
                except:
                    reviewr_name = ""

                try:
                    skin_type_db = []
                    skin_tone_db = []
                    skin_gomin_db = []

                    skin_information = [skin_info["mrkNm"] for skin_info in review_response.json()["gdasList"][each_review]["addInfoNm"]]
                        
                    for skin in skin_information:
                        
                        if skin == "민감성":
                            if len(skin_type_db) == 0:
                                skin_type_db.append(skin)
                            else:
                                skin_gomin_db.append(skin)
                        
                        elif skin[-1] == "성":
                            skin_type_db.append(skin)

                        elif skin[-1] == "톤":
                            skin_tone_db.append(skin)

                        else:
                            skin_gomin_db.append(skin)


                    skin_type_data = " | ".join(skin_type_db)
                    skin_tone_data = " | ".join(skin_tone_db)
                    skin_gomin_data = " | ".join(skin_gomin_db)
                    
                except :
                    skin_type_data = ""
                    skin_tone_data = ""
                    skin_gomin_data = ""

                try:    
                    score = review_response.json()["gdasList"][each_review]["gdasScrVal"]/2
                except:
                    score = 0.0

                try:
                    skin_type_info = " | ".join([skin_type["evalAnsCont"] for skin_type in review_response.json()["gdasList"][each_review]["evalList"]]),
                except:
                    skin_type_info = ""

                try:
                    review = review_response.json()["gdasList"][each_review]["gdasCont"]
                except TypeError:
                    review = ""


                review_output = {

                    "00.상품코드" : prd_no,
                    "01.리뷰어 닉네임" : reviewr_name,
                    "02_1.피부 타입" : skin_type_data,
                    "02_2.피부 톤" : skin_tone_data,
                    "02_3.피부 고민" : skin_gomin_data,
                    "03.리뷰 평점" : score,
                    "04.피부타입 | 피부고민 | 자극도" : skin_type_info,
                    "05.리뷰" : review
                }
                review_result.append(review_output)

        continue
        
#파일명
review_result_df = pd.DataFrame(review_result)
review_result_df.to_csv("review_data_cate_test.csv")
end_time = datetime.now()
print(end_time-start_time)